# Modeling Matching Kidneys as a Network Problem and Solving it by calling Gurobi

A relatively recent development in organ donations (specically kidneys) is that of Kidney exchanges.  If Betsy needs a kidney and her spouse John can donate one but cannot donate to Betsy (say, because they have different blood types) then they can join a kidney exchange.  If another couple (say Jim and Tracy) in the exchange have the same problem but Jim, who needs the kidney, is compatible with Betsy and Tracyis compatible with John then they can make a "swap".  Betsy can get Tracy's kidney and Jim will get John's.  They can be matched.Such types of transfers can also happen in cycles of more than just two pairs.  It could be that we have three pairs (A,B), (C,D) and (E,F) where A gets D's kidney C gets F's and E gets B's. What is feasible depends on the compatibility of the pairs.  

Somewhat simplifying this reality, the problemcan be viewed as a network where each input stream models pairs of a certain type (say, all pairs wherethe person-in-need has blood type Q and the donor has blood-type W). In the network, there are four input types(1,2,3,4). There is a finite number of matching types. A matching is taking of one job from each of the input streams that participate in this matching.  You can either matching a pair of type 1 with a pair of type 2 or match three pairs of types 2, 3 and 4.

A match of type 1, when executed, means that a type-1 pair and a type-2 pairs are taken one from each of their queues, the kidney's are exchanges and the two pairs leave immediately. A matching of type 2,means this happens with one pair of each of the 3 queues.  Thus, using a type-2 matching means serving three pairs while using a type-1 matching means only serving two. Notice that if you have only 100 type-2 pairs arriving per year, you cannot match more than a 100 type-1pairs because these need type 2 pairs.

# Model

Formulate a network model with J input types,I possible matchings and a matrix A which specifes feasible matchings.</br>
Let $\lambda_{1}$,...,$\lambda_{J}$ be the arrival rates of the different inputs types (think of $\lambda_{i}$ the number of type-i pairs arriving to the exchange per year).Let V1,...,VJ be  the  values.  That is,Vi is  the  value  of  serving  a  type-i pair  (thinks  of  this  as  a quantifer of the improvement of quality of life for this pair from being matched and getting a healthy kidney)

Demote arrival rate for matching $1,2,...,I$ as $t_{i}$ 

$max$ $\sum_{j}V_{j}\sum_{i}A_{ij}t_{i}$

$s.t.$ $\sum_{j}A_{ij}t_{i}\leq\lambda_{j,}\forall j\in\{1,2,...,J\}$

$t_{i}\geq0,\forall i\in\{1,2,...,I\}$

# Solving by Gurobi 

Test Case \lambda= (100;120;80;80),v = (10;5;5;4)

In [4]:
from gurobipy import *

# create a new model
myModel = Model( "Online-HW1" )

#-------.dat-----------
# there are 71 papers and 21 referees
noType = 4
noMatching = 2

#parameter: rate
rate = [100,120,80,80]

#parameter: value
v=[10,5,5,4]

#parameter: assignment
A=[[1,1,0,0],[0,1,1,1]]

#-------.mod-----------
# create decision variables and store them in the array myVars
myVars =[]
for i in range (noMatching):
    myVars.append(1)

for i in range(noMatching):
    curVar = myModel.addVar(vtype = GRB.CONTINUOUS ,name= "x"+str(i),lb=0.0)
    myVars[i] = curVar
# integrate decision variables into the model
myModel.update()

# create a linear expression for the objective
objExpr = LinExpr()
for j in range(noType):
    for i in range(noMatching):
        curVar = myVars[i]
        objExpr += v[j]* A[i][j] * curVar
myModel.setObjective( objExpr , GRB.MAXIMIZE )


# create constraints
for j in range(noType):
    constExpr = LinExpr()
    for i in range(noMatching):
        curVar = myVars[i]
        constExpr += A[i][j] * curVar
    myModel.addConstr( lhs = constExpr, sense = GRB.LESS_EQUAL , rhs = rate[j], \
                       name = "type"+str(j))

# integrate objective and constraints into the model
myModel.update()

# write the model in a file to make sure it is constructed correctly
myModel.write( filename = "HW1.lp" )

# optimize the model
myModel.optimize()
# print( "\nOptimal Solution:" )
allVars = myModel.getVars()

# output result
for curVar in allVars:
    print(curVar.x)


Optimize a model with 4 rows, 2 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 1e+02]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.6200000e+03   6.000000e+01   0.000000e+00      0s
       1    1.7800000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.780000000e+03
100.0
20.0
